In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV



df_gols_marcados = pd.read_csv('Gols_Marcados.csv')
df_gols_sofridos = pd.read_csv('Gols_Sofridos.csv')
df_jogadores = pd.read_csv('Jogadores.csv')
df_jogos = pd.read_csv('Jogos.csv')
df_escalacoes = pd.read_csv('Escalacoes.csv')



files = ['Gols_Marcados.csv', 'Gols_Sofridos.csv', 'Jogadores.csv', 'Jogos.csv', 'Escalacoes.csv']
dataframes = []
for file in files:
    dataframes.append(pd.read_csv(file))



for df in dataframes:
    print(df.head())





In [ ]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

for coluna in ['JOGADOR_M', 'ASSISTENCIA_M', 'JOGADOR_S', 'ASSISTENCIA_S', 'SETOR_M', 'AREA_M', 'ESTILO_M',
                'SETOR_S', 'AREA_S', 'ESTILO_S', 'DIA-SEMANA_M', 'DIA-SEMANA_S']:
    if coluna in df.columns:
        if df[coluna].dtype == object:
            df[coluna] = label_encoder.fit_transform(df[coluna])
        else:
            encoded_data = one_hot_encoder.fit_transform(df[coluna].values.reshape(-1, 1))
            encoded_df = pd.DataFrame(encoded_data.toarray(), columns=[f"{coluna}_{cat}" for cat in one_hot_encoder.categories_[0]])  # Use toarray() to get dense array
            df = df.join(encoded_df)


print(df.head())


In [54]:
scaler = StandardScaler()
for coluna in ['JOGADOR_M', 'NUM CAMISA_M', 'PLACAR_M', 'PUBLICO', 'RENDA', 'MINUTO_S',
                'NUM CAMISA_S', 'PLACAR_S', 'JOGOS', 'VITÓRIAS', 'EMPATES', 'DERROTAS',
                'DESEMPENHO (%)', 'GOLS', 'ASSISTÊNCIAS', 'TITULAR', 'RESERVA', 'SUBSTITUÍDO',
                'CAPITÃO', 'CA', 'CV', 'BRASILEIRO', 'PAULISTA', 'COPA DO BRASIL', 'LIBERTADORES',
                'SULAMERICANA', 'AMISTOSOS', 'GOLS NORTE', 'GOLS SUL', 'GOLS 1-TEMPO', 'GOLS 2-TEMPO',
                '2 GOLS', 'HAT-TRICK', 'GOLS DE PÊNALTI', 'PÊNALTIS PERDIDOS', 'GOLS DE FALTA',
                'DENTRO DA ÁREA', 'FORA DA ÁREA', 'PÉ DIREITO', 'PÉ ESQUERDO', 'CABEÇA', 'GOLS CONTRA',
                'GOLS (OUTROS ESTILOS)']:
 df_to_modify = dataframes[0].copy()


In [ ]:
X = df.drop(['GOL_M', 'JOGO', 'LINK_M', 'LINK_S', 'DATA_M', 'DATA_S'], axis=1)
y = df['GOL_M']

In [ ]:
selector = SelectKBest(chi2, k=30)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
X = df[selected_features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
param_grid = {'fit_intercept': [True, False],
              'normalize': [True, False]}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

In [ ]:
model = LinearRegression()

In [ ]:
param_grid = {'fit_intercept': [True, False],
              'normalize': [True, False]}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Gols Reais')
plt.ylabel('Gols Preditos')
plt.title('Gols Reais vs Gols Preditos')
plt.show()